https://www.1point3acres.com/bbs/thread-1089011-1-1.html


### Part 1 

prompt: Stripe in Brazil is obliged to register customer's transactions for each merchant with the central bank as an aggregated unit per day.
These are called receivables. A receivable is identified by 3 identifiers:
* merchant_id (String): The id of the merchant on Stripe side.
* card_type (String): The type of the card used for the transaction (e.g. Visa)
* payout_date (String): String date of the funds available to the merchant by Stripe.
A payment transaction in Stripe API can be represented as the following object:
```
Transaction {
    string customer_id
    string merchant_id
    string payout_date
    string card_type
    int amount
}
```
Implement register_receivables function that takes a string in CSV format
where each line represents a transaction and returns the registered aggregated receivables using the rules above.
Print the returned receivables to console using the format below.
Feel free to parse the CSV using a standard or a 3rd party library or implement it yourself.
You can assume the following about the input:
* The first line of the input is a header. The header is always the same so it can be ignored or hardcoded
* You can assume that the input has
already been read from a file and checked for correctness
* No data fields in this file will include commas or whitespace surrounding the field values.
You can also assume the following about the output:
* The first line of the output is the header. The header is always the same so it can be hardcoded
* Order of the output does not matter
Example input 1:
```
customer_id,merchant_id,payout_date,card_type,amount
cust1,merchantA,2021-12-30,Visa,150
cust2,merchantA,2021-12-30,Visa,200
cust3,merchantB,2021-12-31,MasterCard,300
cust4,merchantA,2021-12-30,Visa,50
```
Output 1:
```
merchant_id,card_type,payout_date,amount
merchantA,Visa,2021-12-30,400
merchantB,MasterCard,2021-12-31,300
```
Example input 2:
```
customer_id,merchant_id,payout_date,card_type,amount
cust1,merchantA,2021-12-29,MasterCard,50
cust2,merchantA,2021-12-29,Visa,150
cust3,merchantB,2021-12-31,Visa,300
cust4,merchantB,2021-12-29,MasterCard,200
```
Output 2
```
merchant_id,card_type,payout_date,amount
merchantA,MasterCard,2021-12-29,50
merchantA,Visa,2021-12-29,150
merchantB,Visa,2021-12-31,300
merchantB,MasterCard,2021-12-29,200
```


#### Solution 1: use pandas

In [28]:
# pip install pandas

In [56]:
import pandas as pd
def register_receivables_part1_pd(file_path):

    df = pd.read_csv(file_path)
    result_df = df.groupby(['merchant_id', 'card_type', 'payout_date']).amount.sum().reset_index()
    result_df.columns = ['merchant_id', 'card_type', 'payout_date', 'amount']

    return result_df
    

In [58]:
print(register_receivables_part1_pd('transaction_data/part1_transactions1.csv'))
print(register_receivables_part1_pd('transaction_data/part1_transactions2.csv'))

  merchant_id   card_type payout_date  amount
0   merchantA        Visa  2021-12-30     400
1   merchantB  MasterCard  2021-12-31     300
  merchant_id   card_type payout_date  amount
0   merchantA  MasterCard  2021-12-29      50
1   merchantA        Visa  2021-12-29     150
2   merchantB  MasterCard  2021-12-29     200
3   merchantB        Visa  2021-12-31     300


#### Solution 2: basic data structures

In [ ]:
import csv
from collections import defaultdict

In [123]:
def register_receivables_part1(file_path, output_file):

    # dictionary to store the aggregated data
    grouped_data = defaultdict(int)

    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader) # skip header and store the header

    #    header: customer_id,merchant_id,payout_date,card_type,amount
        for row in reader:
            customer_id, merchant_id, payout_date, card_type, amount = row[0], row[1], row[2], row[3], int(row[4])
            grouped_data[(merchant_id, card_type, payout_date)] += amount

    # # display
    # for (merchant_id, card_type, payout_date), amount in grouped_data.items():
    #     print(f"merchant_id: {merchant_id}, card_type: {card_type}, payout_date: {payout_date}, amount: {amount}")

    # convert the dictionary to a list of lists (CSV-ready format)
    csv_data = [['merchant_id', 'card_type', 'payout_date', 'amount']]
    csv_data.extend([merchant_id, card_type, payout_date, amount] for (merchant_id, card_type, payout_date), amount in grouped_data.items()) 

    # write to CSV
    with open(output_file, mode = 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)
        
    return csv_data

In [124]:
print(register_receivables_part1('transaction_data/part1_transactions1.csv', 'transaction_data/part1_output_aggregated_data1.csv'))
print(register_receivables_part1('transaction_data/part1_transactions2.csv', 'transaction_data/part1_output_aggregated_data2.csv'))

[['merchant_id', 'card_type', 'payout_date', 'amount'], ['merchantA', 'Visa', '2021-12-30', 400], ['merchantB', 'MasterCard', '2021-12-31', 300]]
[['merchant_id', 'card_type', 'payout_date', 'amount'], ['merchantA', 'MasterCard', '2021-12-29', 50], ['merchantA', 'Visa', '2021-12-29', 150], ['merchantB', 'Visa', '2021-12-31', 300], ['merchantB', 'MasterCard', '2021-12-29', 200]]


### Part 2/3:

In Brazil, settlement times can take up to 30 days for domestic card transaction. i.e. A merchant selling items online
will receive their money from a customer after a month of selling an item. This has created a need where merchants
are looking for ways to receive their money faster.
Per the regulations, merchants can sell their receivables to a financial institution. The financial institution
will pay the funds to the merchant earlier and receive the funds from Stripe instead on the payout date.
An agreement between the merchant and a financial institution is called a contract. Stripe is obliged to respect
those contracts and update the registered receivables. Each contract is mapped to one receivable based on the
same 3 identifiers as above:
* merchant_id (String): The id of the merchant on Stripe side.
* card_type (String): The type of the card used for the transaction (e.g. Visa)
* payout_date (String): String date of the funds available to the merchant by Stripe.
A contract sent to Stripe is represented as follows:
```
Contract {
    string contract_id
    string merchant_id
    string payout_date
    string card_type
    integer amount
}
```
Implement update_receivables function that takes the list of registered receivables from part 1 and additional parameter of list of contracts.
The result should be the updated list of receivables.
For each contract, a receivable should be created for the contract id, and the corresponding merchant receivable should be removed.
Example input 1:
Transactions:
```
customer_id,merchant_id,payout_date,card_type,amount
cust1,merchantA,2022-01-05,Visa,300
cust2,merchantA,2022-01-05,Visa,200
cust3,merchantB,2022-01-06,MasterCard,1000
```
Contracts:
```
contract_id,merchant_id,payout_date,card_type,amount
contract1,merchantA,2022-01-05,Visa,500
```
=> update_receivables(registered_receivables, input_contracts)
Output 1:
```
id,card_type,payout_date,amount
contract1,Visa,2022-01-05,500
merchantB,MasterCard,2022-01-06,1000
```
Example input 2:
Transactions:
```
customer_id,merchant_id,payout_date,card_type,amount
cust1,merchantA,2022-01-07,Visa,500
cust2,merchantA,2022-01-07,Visa,250
cust3,merchantB,2022-01-08,MasterCard,1250
cust4,merchantC,2022-01-09,Visa,1500
```
Contracts:
```
contract_id,merchant_id,payout_date,card_type,amount
contract1,merchantA,2022-01-07,Visa,750
contract2,merchantC,2022-01-09,Visa,1500
```
=> update_receivables(registered_receivables, input_contracts)
Output 2:
```
id,card_type,payout_date,amount
contract1,Visa,2022-01-07,750
contract2,Visa,2022-01-09,1500
merchantB,MasterCard,2022-01-08,1250
```
*/

### Part 3:
It is possible to have fully covered, or partially covered. So keep the original records with subtraction (even 0), add new records with contracts

In [140]:
import pandas as pd

def update_receivables_pd(registered_receivables, input_contracts):

    df_tran = pd.read_csv(registered_receivables)
    df_cont = pd.read_csv(input_contracts)
    df_cont.columns = ["cont_amount" if col == "amount" else col for col in df_cont.columns]

    # aggregate the transaction amount - receivables
    df_agg = df_tran.groupby(['merchant_id', 'card_type', 'payout_date']).amount.sum().reset_index()
    df_agg.columns = ['merchant_id', 'card_type', 'payout_date', 're_amount']
    
    
    # combine receivables and contracts
    df_total = df_agg.merge(df_cont, how = 'left', on = ['merchant_id', 'card_type', 'payout_date'])
    
    # # if the receivable amount are all covered by the contract amount
    # ## update the merchant_id to contract_id, also remove the values from contract columns
    # df_total.loc[df_total['re_amount'] == df_total['cont_amount'], 'merchant_id'] = df_total['contract_id']
    # df_total.loc[df_total['re_amount'] == df_total['cont_amount'], 'contract_id'] = None
    # df_total.loc[df_total['re_amount'] == df_total['cont_amount'], 'cont_amount'] = None

    # if the receivable amount are partially coverted by the contract amount
    df_total.loc[df_total['re_amount'] >= df_total['cont_amount'], 're_amount'] = df_total['re_amount'] - df_total['cont_amount']
    # add all contracts (partially covered) to the total dataframe as new records 
    # fully covered receivables' contracts are being modified to None before
    df_processed = df_total[df_total['contract_id'].notna()][['contract_id', 'card_type', 'payout_date', 'cont_amount']]
    df_processed.columns = ['merchant_id', 'card_type', 'payout_date', 're_amount']

    df_result = pd.concat([df_total[['merchant_id', 'card_type', 'payout_date', 're_amount']], df_processed], axis = 0)
    df_result.columns = ["id" if col == "merchant_id" else col for col in df_result.columns]
    df_result.columns = ["amount" if col == "re_amount" else col for col in df_result.columns]

    return df_result
    

In [141]:
print(update_receivables_pd('transaction_data/part2_transactions1.csv', 'transaction_data/part2_contracts1.csv'))
print(update_receivables_pd('transaction_data/part2_transactions2.csv', 'transaction_data/part2_contracts2.csv'))

          id   card_type payout_date  amount
0  merchantA        Visa  2022-01-05     0.0
1  merchantB  MasterCard  2022-01-06  1000.0
2  merchantC  MasterCard  2022-01-07   100.0
0  contract1        Visa  2022-01-05   500.0
2  contract2  MasterCard  2022-01-07   500.0
          id   card_type payout_date  amount
0  merchantA        Visa  2022-01-07     0.0
1  merchantB  MasterCard  2022-01-08  1250.0
2  merchantC        Visa  2022-01-09     0.0
0  contract1        Visa  2022-01-07   750.0
2  contract2        Visa  2022-01-09  1500.0


In [142]:
def update_receivables_part2(tran_file_path, cont_file_path, output_file):

    # dictionary to store the raw data
    grouped_data = defaultdict(int)

    # read transaction data and aggregate
    with open(tran_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader) # skip header and store the header

    #    header: customer_id,merchant_id,payout_date,card_type,amount
        for row in reader:
            customer_id, merchant_id, payout_date, card_type, re_amount = row[0], row[1], row[2], row[3], int(row[4])
            grouped_data[(merchant_id, card_type, payout_date)] += re_amount

    # read contract data and store
    with open(cont_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)

        for row in reader:
            contract_id, merchant_id, payout_date, card_type, cont_amount = row[0], row[1], row[2], row[3], int(row[4])
            if (merchant_id, card_type, payout_date) in grouped_data:
                new_amount = grouped_data[(merchant_id, card_type, payout_date)] - cont_amount

                grouped_data[(merchant_id, card_type, payout_date)] = new_amount
                grouped_data[(contract_id, card_type, payout_date)] = cont_amount

    # # # display
    # for (id, card_type, payout_date), amount in grouped_data.items():
    #     print(f"id: {id}, card_type: {card_type}, payout_date: {payout_date}, amount: {amount}")

    # convert the dictionary to a list of lists (CSV-ready format)
    csv_data = [['id', 'card_type', 'payout_date', 'amount']]
    csv_data.extend([id, card_type, payout_date, amount] for (id, card_type, payout_date), amount in grouped_data.items()) 

    # write to CSV
    with open(output_file, mode = 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)
        
    return csv_data

In [143]:
update_receivables_part2('transaction_data/part2_transactions1.csv', 'transaction_data/part2_contracts1.csv', 'transaction_data/part2_output_results1.csv')

[['id', 'card_type', 'payout_date', 'amount'],
 ['merchantA', 'Visa', '2022-01-05', 0],
 ['merchantB', 'MasterCard', '2022-01-06', 1000],
 ['merchantC', 'MasterCard', '2022-01-07', 100],
 ['contract1', 'Visa', '2022-01-05', 500],
 ['contract2', 'MasterCard', '2022-01-07', 500]]